In [2]:
import certifi
import time
import sys
import datetime
import os

from variables import MONGO_URL
import json

import numpy
from sportmonks import SportMonksAPI
from pymongo import MongoClient
import pandas as pd
import boto3
from fixture_formatter import FixtureFormatter
s3 = boto3.client('s3')
bucket = "footyamigo-fixtures"

formatter = FixtureFormatter()

ca = certifi.where()

client = MongoClient(MONGO_URL, tlsCAFile=ca)
database = client.footyamigo
fixtures_table = database.fixtures

picks_table = database.picks
nan = None

# for fixture_id in picks_table.distinct("fixture_id", {"home_name": {"$exists": False}}):
#     fixture = fixtures_table.find_one({"fixture_id": fixture_id}, {
#                                       "home_name": 1, "away_name": 1})
#     res= picks_table.update_many({'fixture_id': fixture_id}, {"$set": {
#                             "home_name": fixture["home_name"], "away_name": fixture["away_name"]}})
#     print(res)


In [11]:
fixtures_table.create_index(list({ "fixture_name": 'text', "league_name": 'text', "country_name": 'text' }.items()), default_language='english')( )

TypeError: 'str' object is not callable

In [1]:
import pymongo

In [9]:
db.collection.createIndex( { "fieldA.$**" : 1 } )

dict_items([('fixture_name', 'text'), ('league_name', 'text'), ('country_name', 'text')])

In [4]:
fixtures_table.create_index(list({ "home.$**" : 1  }.items()), default_language='english')

'home.$**_1'

In [25]:
fields="""
played_home
played_away
played_overall
o15_goals_overall_per
o25_goals_overall_per
o35_goals_overall_per
o85_corners_overall_per
o95_corners_overall_per
o105_corners_overall_per
o05_goals_1h_overall_per
total_goals_overall_avg
corners_total_overall_avg
"""

In [26]:
fields = fields.splitlines()

In [27]:
fields = ["home."+ x for x in fields] + ["away."+ x for x in fields] 

In [28]:
fields

['home.o15_goals_overall_per',
 'home.o25_goals_overall_per',
 'home.o35_goals_overall_per',
 'home.o85_corners_overall_per',
 'home.o95_corners_overall_per',
 'home.o105_corners_overall_per',
 'home.o05_goals_1h_overall_per',
 'home.total_goals_overall_avg',
 'home.corners_total_overall_avg',
 'home.played_home',
 'home.played_away',
 'home.played_overall',
 'away.o15_goals_overall_per',
 'away.o25_goals_overall_per',
 'away.o35_goals_overall_per',
 'away.o85_corners_overall_per',
 'away.o95_corners_overall_per',
 'away.o105_corners_overall_per',
 'away.o05_goals_1h_overall_per',
 'away.total_goals_overall_avg',
 'away.corners_total_overall_avg',
 'away.played_home',
 'away.played_away',
 'away.played_overall']

In [43]:
indexes = """
home.played_overall
away.played_overall
home.o25_goals_overall_per
home.total_goals_overall_avg
away.total_goals_overall_avg
probability.o25_goals_probability
home.scored_o05_home_per
away.scored_o05_away_per
""".split()

In [44]:
indexes = [(x, 1) for x in indexes]

In [45]:
indexes

[('home.played_overall', 1),
 ('away.played_overall', 1),
 ('home.o25_goals_overall_per', 1),
 ('home.total_goals_overall_avg', 1),
 ('away.total_goals_overall_avg', 1),
 ('probability.o25_goals_probability', 1),
 ('home.scored_o05_home_per', 1),
 ('away.scored_o05_away_per', 1)]

In [32]:
len(indexes)

32

In [46]:
fixtures_table.create_index(indexes, default_language='english', name="o25_goals")

'o25_goals'

In [85]:
from itertools import product
with open("hit_rates.csv", "a") as writer:
  for a, b ,c , d ,e,f ,g, h in product(range(1,40), range(1,40), range(50,100), range(2,15), range(2,15), range(40,100), range(50,100), range(50,100) ):
    condition = {
      "$and": [
        {
          "home.played_overall": {
            "$gte": a,
            "$lte": 60
          }
        },
        {
          "away.played_overall": {
            "$gte": b,
            "$lte": 60
          }
        },
        {
          "home.o25_goals_overall_per": {
            "$gte": c,
            "$lte": 100
          }
        },
        {
          "home.total_goals_overall_avg": {
            "$gte": d,
            "$lte": 15
          }
        },
        {
          "away.total_goals_overall_avg": {
            "$gte": e,
            "$lte": 15
          }
        },
        {
          "probability.o25_goals_probability": {
            "$gte": f,
            "$lte": 100
          }
        },
        {
          "home.scored_o05_home_per": {
            "$gte": g,
            "$lte": 100
          }
        },
        {
          "away.scored_o05_away_per": {
            "$gte": h,
            "$lte": 100
          }
        }
      ]
    }

    fixtures_found = fixtures_table.count_documents(condition)
    # condition.update(outcome_condition)
    hits_found = fixtures_table.count_documents({**condition, "result.o25_goals": True})
    hit_rate = round(hits_found/(fixtures_found or 1)*100, 2)
    print(fixtures_found, hits_found, hit_rate, [a,b,c,d,e,f,g,h])
    writer.write(f"{fixtures_found}, {hits_found}, {hit_rate}, {[a,b,c,d,e,f,g,h]}")

13200 7705 58.37 [1, 1, 50, 2, 2, 40, 50, 50]
8640 5026 58.17 [1, 1, 50, 2, 2, 40, 50, 51]
8640 5026 58.17 [1, 1, 50, 2, 2, 40, 50, 52]
8640 5026 58.17 [1, 1, 50, 2, 2, 40, 50, 53]
8623 5016 58.17 [1, 1, 50, 2, 2, 40, 50, 54]
8576 4989 58.17 [1, 1, 50, 2, 2, 40, 50, 55]
8477 4928 58.13 [1, 1, 50, 2, 2, 40, 50, 56]


KeyboardInterrupt: 

72.95


In [71]:
fixtures_found

732

In [72]:
hits_found

534

In [63]:
fixtures_found

534

In [89]:
from itertools import product
for i in range(100):
  condition = {
      "$and": [
        {
          "home.played_overall": {
            "$gte": 1,
            "$lte": 60
          }
        },
        {
          "away.played_overall": {
            "$gte": 1,
            "$lte": 60
          }
        },
        {
          "home.o25_goals_overall_per": {
            "$gte": 50,
            "$lte": 100
          }
        },
        {
          "home.total_goals_overall_avg": {
            "$gte": 3,
            "$lte": 15
          }
        },
        {
          "away.total_goals_overall_avg": {
            "$gte": 3,
            "$lte": 15
          }
        },
        {
          "probability.o25_goals_probability": {
            "$gte": i,
            "$lte": 100
          }
        },
        {
          "home.scored_o05_home_per": {
            "$gte": 60,
            "$lte": 100
          }
        },
        {
          "away.scored_o05_away_per": {
            "$gte": 10,
            "$lte": 100
          }
        }
      ]
    }

  fixtures_found = fixtures_table.count_documents(condition)
  # condition.update(outcome_condition)
  hits_found = fixtures_table.count_documents({**condition, "result.o25_goals": True})
  hit_rate = round(hits_found/(fixtures_found or 1)*100, 2)
  print(fixtures_found, hits_found, hit_rate, i)
  #writer.write(f"{fixtures_found}, {hits_found}, {hit_rate}, {[a,b,c,d,e,f,g,h]}")

9581 6244 65.17 0
9581 6244 65.17 1
9581 6244 65.17 2
9581 6244 65.17 3
9581 6244 65.17 4
9581 6244 65.17 5
9581 6244 65.17 6
9581 6244 65.17 7
9581 6244 65.17 8
9581 6244 65.17 9
9581 6244 65.17 10
9581 6244 65.17 11
9581 6244 65.17 12
9581 6244 65.17 13
9581 6244 65.17 14
9581 6244 65.17 15
9581 6244 65.17 16
9581 6244 65.17 17
9581 6244 65.17 18
9581 6244 65.17 19
9581 6244 65.17 20
9581 6244 65.17 21
9581 6244 65.17 22
9581 6244 65.17 23
9581 6244 65.17 24
9581 6244 65.17 25
9581 6244 65.17 26
9580 6244 65.18 27
9579 6243 65.17 28
9579 6243 65.17 29
9578 6243 65.18 30
9578 6243 65.18 31
9575 6241 65.18 32
9571 6241 65.21 33
9568 6240 65.22 34
9562 6236 65.22 35
9552 6232 65.24 36
9545 6227 65.24 37
9531 6219 65.25 38
9517 6215 65.3 39
9498 6205 65.33 40
9468 6191 65.39 41
9428 6172 65.46 42
9380 6149 65.55 43
9327 6123 65.65 44
9267 6090 65.72 45
9186 6048 65.84 46
9100 6011 66.05 47
9006 5963 66.21 48


KeyboardInterrupt: 